# Solve RCPSP by constraint programming

One of the main class of methods to solve discrete optimization problem is Constraint programming [(CP)](https://en.wikipedia.org/wiki/Constraint_programming). CP solvers explore the state variables and propagate constraints in a clever way. Using constraint programming, users declaratively state the constraints on the feasible solutions for a set of decision variables. Constraints specify the properties of a solution to be found.
In discrete-optimization library, [minizinc](https://www.minizinc.org/) declarative language and its [python api](https://minizinc-python.readthedocs.io/en/latest/) is used extensively. However it is quite easy to use other modeling library such as [Cpmpy](https://github.com/CPMpy/cpmpy/tree/master/cpmpy) in the future.
Some constraint programming models are stored in discrete_optimization/knapsack/minizinc folder.

In this notebook, we'll use the [chuffed](https://github.com/chuffed/chuffed#description) solver which is a state of the art lazy clause solver. 

We assume that you have already been introduced to RCPSP problems thanks to this [notebook](RCPSP%20%231%20Introduction.ipynb).

## Prerequisites


Concerning the python kernel to use for this notebook:
- If running locally, be sure to use an environment with discrete-optimization and minizinc.
- If running on colab, the next cell does it for you.
- If running on binder, the environment should be ready.


In [ ]:
# On Colab: install the library
on_colab = "google.colab" in str(get_ipython())
if on_colab:
    import importlib
    import os
    import sys  # noqa: avoid having this import removed by pycln

    !{sys.executable} -m pip install -U pip

    # uninstall google protobuf conflicting with ray and sb3
    ! pip uninstall -y protobuf

    # install dev version for dev doc, or release version for release doc
    !{sys.executable} -m pip install git+https://github.com/airbus/discrete-optimization@master#egg=discrete-optimization

    # be sure to load the proper cffi (downgraded compared to the one initially on colab)
    import cffi

    importlib.reload(cffi)

    # install and configure minizinc
    !curl -o minizinc.AppImage -L https://github.com/MiniZinc/MiniZincIDE/releases/download/2.6.3/MiniZincIDE-2.6.3-x86_64.AppImage
    !chmod +x minizinc.AppImage
    !./minizinc.AppImage --appimage-extract
    os.environ["PATH"] = f"{os.getcwd()}/squashfs-root/usr/bin/:{os.environ['PATH']}"
    os.environ["LD_LIBRARY_PATH"] = (
        f"{os.getcwd()}/squashfs-root/usr/lib/:{os.environ['LD_LIBRARY_PATH']}"
    )

### Imports

In [ ]:
import logging

import matplotlib.pyplot as plt
import nest_asyncio

from discrete_optimization.datasets import fetch_data_from_psplib
from discrete_optimization.rcpsp.rcpsp_parser import get_data_available, parse_file
from discrete_optimization.rcpsp.rcpsp_utils import plot_ressource_view
from discrete_optimization.rcpsp.solver.cp_solvers import (
    CP_RCPSP_MZN,
    CPSolverName,
    ParametersCP,
)

# patch asyncio so that applications using async functions can run in jupyter
nest_asyncio.apply()

# set logging level
logging.basicConfig(level=logging.INFO)

### Download datasets

If not yet available, we import the datasets from [psplib](https://www.om-db.wi.tum.de/psplib/data.html).

In [ ]:
needed_datasets = ["j301_1.sm"]
download_needed = False
try:
    files_available_paths = get_data_available()
    for dataset in needed_datasets:
        if len([f for f in files_available_paths if dataset in f]) == 0:
            download_needed = True
            break
except:
    download_needed = True

if download_needed:
    fetch_data_from_psplib()

In [ ]:
# Parse some rcpsp file
filepath = [f for f in get_data_available() if "j601_1.sm" in f][0]
rcpsp_problem = parse_file(filepath)

## Constraint modelling
The simplest constraint model for RCPSP can be found in the library at this [path](../discrete_optimization/rcpsp/minizinc/rcpsp_single_mode_mzn.mzn) (rcpsp/minizinc/rcpsp_single_mode_mzn.mzn). 

Contrary to the linear programming formulation, no need of binary variable (that were needed to make the formulation linear). The decisions variable are directly the starting time of the task. 
The precedence constraints are then quite trivial to write

$\forall t_1, \forall t_2 \in \text{successors}(t_1), start[t_2]\geq start[t_1]$

About the cumulative constraint, we use the <b>global</b> constraint *cumulative* that is native in minizing language : see [here](https://www.minizinc.org/doc-2.5.5/en/lib-globals.html#index-119) for more details.

## CP Solver 

In [ ]:
solver = CP_RCPSP_MZN(problem=rcpsp_problem, cp_solver_name=CPSolverName.CHUFFED)
solver.init_model()
parameters_cp = ParametersCP.default()
parameters_cp.time_limit = 20
results = solver.solve(parameters_cp=parameters_cp)

For this instance, the solving is very fast (should be faster than when testing the previous Linear Programming isn't it ?). CP is here able to return optimal solution almost instantly.

In [ ]:
fig, ax = plt.subplots(1)
ax.plot([x[1] for x in results.list_solution_fits], marker="o")
ax.set_ylabel("- makespan")
ax.set_xlabel("# solution found")
plt.show()

In [ ]:
# Retrieve the best_solution, fit found by the solver.
best_solution, fit = results.get_best_solution_fit()
# Print fitness
print(fit)
# Check if the solution satisfies the constraints
print("Satisfaction :", rcpsp_problem.satisfy(best_solution))
# Evaluation :
print("Evaluation :", rcpsp_problem.evaluate(best_solution))

### Plot the solution

In [ ]:
plot_ressource_view(rcpsp_model=rcpsp_problem, rcpsp_sol=best_solution)

Constraint programming is the state of the art method to solve scheduling problems like $RCPSP$. It is therefore not surprising to get fast and good solutions for this limited-size instance. 
Many other models have been developed in the library to tackle complex variants of scheduling problems.
Few characteristics of such scheduling problems are the following : 
- variable resource availability
- multimode task execution (for example a task can be fast+highly resource demanding or slow+low resource demanding)
- preemptive scheduling : ability to pause and resume task.
